## Class to process page 2 of the survey

In [36]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
plt.rcParams['figure.figsize'] = (10, 10)
from utils import *
from define import *
# import define

In [37]:
class Page2Processor:
    def __init__(self, xml_path):
        df = xml2csv(xml_path)

        self.all_checkbox = df.copy()

        self.all_checkbox_numpy =np.array(self.all_checkbox[['xmin', 'ymin', 'xmax', 'ymax']])
        question_of_row = [row.split('_') for row in self.all_checkbox['class']]
        self.question_of_row = [row[0] if len(row) == 2 else '_'.join(row[:3]) if len(row) == 4 else '_'.join(row[:2]) for row in question_of_row]

        
    def process(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        extractor = CheckboxExtractor()
        checkboxes = extractor.detect_checkbox(gray)    
        tick_list = list(map(lambda x: extractor.is_ticked(gray[x[1]:x[3], x[0]:x[2]]), checkboxes))
        checkboxes = np.hstack([checkboxes, np.array(tick_list).reshape(-1,1)]) # [x1,y1,x2,y2,is_ticked]

        row_ids = []
        for i in range(len(checkboxes)):
            x1,y1,x2,y2,_is_tick = checkboxes[i]
            checkbox_coor = checkboxes[i:i+1, :]
            _iou = np_vec_no_jit_iou(checkbox_coor[:,:4], self.all_checkbox_numpy)
            row_ids += [np.argmax(_iou)]
        
        result_df = pd.DataFrame()

        _id_offset = 0

        for _question_id in [18,19,20]:
            question_name = f'{_question_id}'

            _checkbox_ids = list(filter(lambda x: ( str(_question_id) == self.question_of_row[row_ids[x]] ) and (checkboxes[x][4] == 1), range(len(checkboxes))))

            answers = []
            if len(_checkbox_ids) > 0:
                for _checkbox_id in _checkbox_ids:
                    answers += [ANSWER_TEXT_2[str(_question_id)][row_ids[_checkbox_id] - _id_offset]]
            result_df[question_name] = [', '.join(answers)]
            _id_offset += len(ANSWER_TEXT_2[str(_question_id)])
            # if _question_id in [31, 33]:
            #     answers = []
            #     question_name = f'{_question_id}_1'
            #     _checkbox_ids = list(filter(lambda x: ( self.question_of_row[row_ids[x]] == question_name ) and (checkboxes[x][4] == 1), range(len(checkboxes))))
            #     for _checkbox_id in _checkbox_ids:
            #         answers += [ANSWER_TEXT_5[question_name][row_ids[_checkbox_id] - _id_offset]]
            #     result_df[question_name.replace('_', '.')] = [', '.join(answers)]
            #     _id_offset += len(ANSWER_TEXT_5[question_name])




        return result_df

    


In [38]:
processor = Page2Processor('page2.xml')

img = cv2.imread('page2.jpg')
ret = processor.process(img)

ret

xml to csv page2.xml


,18,19,20
0,No,No,"Teacher/coach, Other"
